In [1]:
from IPython.display import HTML
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import urllib
import requests
import random
import re
import numpy as np

In [2]:
df = pd.DataFrame(columns=['ranking', 'url','title', 'region','time','director','actor',
                               '1 week culmulative income','all culmulative income','duration','type','rating'])

In [2]:
def safeRequest(url):
    s = requests.Session()
    s.headers = {
     "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36",
     "Cookie":"uuid_n_v=v1; uuid=8689C6B03C7111EC9BB8473BB5308EAD38570A4642934DBEAF79D397BB45460E; _csrf=cb357b0e621cddb183cffe459dc9b83d3d21f458b101e6720e940ee02cb4dfb2; _lxsdk_cuid=17cd57404b9c8-05ed84795e2244-57b193e-384000-17cd57404b9c8; _lxsdk=8689C6B03C7111EC9BB8473BB5308EAD38570A4642934DBEAF79D397BB45460E; lt=DGck9TomU47uwSjS58jBh4OpWwUAAAAACw8AAEdxyHbcYZawGn7t37XHtZ5TGJvKYVjQmxcw659dS3A47h33j0la2m4cxbeHs2p5Ig; lt.sig=_W1GYDyvumEXPnkFP1g2-UbxlGo; uid=1540656660; uid.sig=r2cunr-_CSLZRJvakXQeqGCu5tE; Hm_lvt_703e94591e87be68cc8da0da7cbd0be2=1635921812,1635921833,1635921884,1635921889; Hm_lpvt_703e94591e87be68cc8da0da7cbd0be2=1635921889; __mta=244096314.1635921884309.1635921884309.1635921889461.2; _lxsdk_s=17ce48af373-a0b-91c-af2||4"
    }
    r = s.request("GET", url=url)
    r.encoding = "utf-8"
    return r.content

In [3]:
# Test request
url = "https://www.maoyan.com/films/1284164"
content = safeRequest(url)
soup = BeautifulSoup(content, 'html.parser')

In [25]:
def getMovieURL():
    url_list = []
    home_url = "https://www.maoyan.com/board/4?offset="
    movie_url = "https://www.maoyan.com"
    flag = 0
    for i in range(0,100,10):
        url=home_url+str(i)
        while len(url_list) != i+10:
            content = safeRequest(url)
            soup = BeautifulSoup(content, 'html.parser')
            res_data=soup.findAll("dd")
            if len(res_data)==0 and flag==0:
                print("please view the page for verification",url)
                flag=1
            else:
                for j in res_data:
                    url_list.append(movie_url+str(j.a.get("href")))
                    flag=0
            time.sleep(random.random()*3)
    return url_list

In [24]:
#Test
len(getMovieURL())

please view the page for verification https://www.maoyan.com/board/4?offset=40


100

In [26]:
def getMName(soup):
    return soup.find('h1',class_="name")

In [20]:
#Test
getMName(soup).string

'少年派的奇幻漂流'

In [27]:
def getMType(soup):
    mtype_data = soup.find('li',class_="ellipsis").findAll('a')
    mtype=[]
    for i in mtype_data:
        mtype.append(re.search(r'(\S+)',i.string).group(0))
    return mtype

In [35]:
#Test
getMType(soup)

['剧情', '喜剧']

In [4]:
def getMDuration(soup):
    tmp = soup.find('div',class_="movie-brief-container").findAll('li')[1].string
    duration = re.search(r'\d{2,3}',tmp).group(0)
    return duration

In [5]:
#Test
getMDuration(soup)

'109'

In [17]:
def getMTR(soup):
    tmp = soup.find('div',class_="movie-brief-container").findAll('li')[2].string
    mtime = re.search(r'.{7,9}\d',tmp).group(0)
    region = re.search(r'\D*上',tmp).group(0)[:-1]
    return mtime, region

In [18]:
#Test
getMTR(soup)

('2000-10-26', '意大利')

In [30]:
def getMCelebrity(soup):
    director_list = []
    actors_list = []

    celebrity_data = soup.find('div',class_="tab-celebrity tab-content")
    director_num_raw, actor_num_raw = celebrity_data.find_all('span',class_="num")[0:2]
    director_num = int(re.search(r'\d+',director_num_raw.string).group(0))
    actor_num = int(re.search(r'\d+',actor_num_raw.string).group(0))
    celebrity_name = celebrity_data.find_all("div",class_="info")[:(director_num+actor_num)]
    for i  in range(director_num):
        director_list.append(re.search(r'\w+.*\w',celebrity_name[i].a.string).group(0))
    for i in range(actor_num):
        actors_list.append(re.search(r'\w+.*\w',celebrity_name[i+director_num].a.string).group(0))
    return director_list, actors_list

In [40]:
#Test
getMCelebrity(soup)

(['文牧野'],
 ['徐峥',
  '周一围',
  '王传君',
  '谭卓',
  '章宇',
  '杨新鸣',
  '王砚辉',
  '贾晨飞',
  '王佳佳',
  '李乃文',
  '龚蓓苾',
  '苇青',
  '邓飞',
  '刘頔',
  '宁浩',
  '岳小军',
  '富冠铭',
  '巴拉特·巴蒂',
  '喜利图',
  '张海艳',
  '朱耕佑',
  '张子贤',
  '任洛敏',
  '杜子蓝',
  '刘兴雷',
  '杜家毅',
  '宁晓志',
  '马千壹',
  '石强',
  '甘昀宸',
  '何李宁',
  '陈翃',
  '肖龙',
  'Chenfei Jia'])

In [31]:
def getMRating(soup):
    return float(re.search(r'\d+.*\d',soup.find("div",class_="star-wrapper").div.get("style")).group(0))/10

In [42]:
#Test
getMRating(soup)

9.6

In [32]:
def getMIncome(soup):
    income_raw = soup.find_all("div",class_="film-mbox-item")
    
    if len(income_raw) == 0:
        income_1week = None
        income_all = None
    else:
        i = len(income_raw)
        if income_raw[i-2].div.string == "暂无":
            income_1week = None
        else:
            income_1week = int(income_raw[i-2].div.string)

        if income_raw[i-1].div.string == "暂无":
            income_all = None
        else:
            income_all = int(income_raw[i-1].div.string)
        
    return income_1week, income_all

In [93]:
#Test
print(getMIncome(BeautifulSoup(safeRequest("https://www.maoyan.com/films/243"), 'html.parser')))
time.sleep(random.random()*3+5)
print(getMIncome(BeautifulSoup(safeRequest("https://www.maoyan.com/films/1297"), 'html.parser')))
time.sleep(random.random()*3+5)
print(getMIncome(BeautifulSoup(safeRequest("https://www.maoyan.com/films/1203"), 'html.parser')))
time.sleep(random.random()*3+5)
print(getMIncome(BeautifulSoup(safeRequest("https://www.maoyan.com/films/1205"), 'html.parser')))
time.sleep(random.random()*3+5)
print(getMIncome(BeautifulSoup(safeRequest("https://www.maoyan.com/films/1200486"), 'html.parser')))

(None, 171507)
(None, None)
(None, 5)
(None, None)
(123812, 310002)


In [45]:
def getMData(ranking, url):
    name_raw = None
    flag = 0
    while name_raw==None :
        content = safeRequest(url)
        soup = BeautifulSoup(content, 'html.parser')
        name_raw = getMName(soup)
        if flag == 0 and name_raw == None:
            print("please view the page for verification",url)
            flag = 1
            time.sleep(random.random()*3)
    name = name_raw.string
    mtype = getMType(soup)
    duration = getMDuration(soup)
    mtime, region = getMTR(soup)
    director_list, actors_list = getMCelebrity(soup)
    rating = getMRating(soup)
    income_1week, income_all = getMIncome(soup)
    df.loc[ranking-1] = [ranking,url,name,region,mtime,director_list,actors_list,
                         income_1week, income_all,duration,mtype,rating]
    print(ranking,name,"is completed")

In [78]:
#Test
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
getMData(1,"https://www.maoyan.com/films/1200486")

1 我不是药神 is completed


In [34]:
def getMDataAll(url_list): 
    ranking = 1
    for url in url_list:
        getMData(ranking, url)
        ranking+=1
        time.sleep(random.random()*3)

In [ ]:
url_list = getMovieURL()
print("url complete")
getMDataAll(url_list)
df.to_csv("data.csv", mode='w', encoding="utf_8_sig", index=None, header=True)